In [1]:
import functions as fn
import pandas as pd
from importlib import reload
from tqdm import tqdm
from termcolor import colored
import matplotlib.pyplot as plt
from ooipy.tools import ooiplotlib as ooiplt
import datetime

In [57]:
df = pd.read_csv('E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/oregon_hydrate.csv',sep = ',')


In [58]:
df.columns

Index(['Unnamed: 0', 'MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG',
       'Heading', 'VesselName', 'IMO', 'CallSign', 'Status', 'LENGTH', 'Width',
       'Draft', 'Cargo', 'TransceiverClass', 'TIMESTAMP UTC',
       'oregon_slope_Lat', 'oregon_slope_Lon', 'oregon_slope_Loc', 'ship_Loc',
       'Southern_hydrate_Lat', 'Southern_hydrate_lon', 'Southern_hydrate_Loc',
       'distance(in km) oregon', 'distance(in km) hydrate', 'Type Code',
       'VESSEL TYPE', 'Group', 'group3'],
      dtype='object')

In [59]:
df['TIMESTAMP UTC']=df['TIMESTAMP UTC'].astype('datetime64[ns]')


In [60]:
df.head()

,Unnamed: 0,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,...,ship_Loc,Southern_hydrate_Lat,Southern_hydrate_lon,Southern_hydrate_Loc,distance(in km) oregon,distance(in km) hydrate,Type Code,VESSEL TYPE,Group,group3
0,0,636092819,2021-01-01T00:17:40,44.56820,-125.62248,11.0,180.6,181.0,NaN,NaN,...,"(44.5682, -125.62248)",44.5691,-125.1481,"(44.5691, -125.1481)",19.40,37.68,NaN,NaN,NaN,NaN
1,1,636092819,2021-01-01T00:19:00,44.56414,-125.62273,11.1,187.9,180.0,NaN,NaN,...,"(44.56414, -125.62273)",44.5691,-125.1481,"(44.5691, -125.1481)",19.29,37.71,NaN,NaN,NaN,NaN
2,2,636092819,2021-01-01T00:20:08,44.56068,-125.62271,11.4,162.9,179.0,NaN,NaN,...,"(44.56068, -125.62271)",44.5691,-125.1481,"(44.5691, -125.1481)",19.19,37.72,NaN,NaN,NaN,NaN
3,3,636092819,2021-01-01T00:22:19,44.55397,-125.62281,10.9,186.6,182.0,NaN,NaN,...,"(44.55397, -125.62281)",44.5691,-125.1481,"(44.5691, -125.1481)",19.01,37.75,NaN,NaN,NaN,NaN
4,4,636092819,2021-01-01T00:24:00,44.54881,-125.62310,10.8,177.9,180.0,NaN,NaN,...,"(44.54881, -125.6231)",44.5691,-125.1481,"(44.5691, -125.1481)",18.91,37.81,NaN,NaN,NaN,NaN


In [61]:
df['TIMESTAMP UTC'].min()

Timestamp('2017-01-01 09:48:46')

In [62]:
df['TIMESTAMP UTC'].max()

Timestamp('2022-06-30 23:59:29')

In [63]:
df['VESSEL TYPE'].unique()

array([nan, 'Cargo, all ships of this type', 'Pleasure Craft', 'Fishing',
       'Sailing', 'Passenger, all ships of this type',
       'Tanker, all ships of this type',
       'Other Type, all ships of this type',
       'Cargo, Hazardous category A', 'Towing',
       'Cargo, Hazardous category C',
       'Other Type, no additional information',
       'Cargo, No additional information', 'Spare - Local Vessel',
       'Tanker, No additional information', 'Cargo, Hazardous category B',
       'Cargo, Hazardous category D', 'Tanker, Hazardous category B',
       'Tanker, Hazardous category A', 'Tug', 'Not available (default)',
       'Wing in ground (WIG), all ships of this type',
       'Other Type, Hazardous category A', 'Tanker, Hazardous category D',
       'Military ops',
       'Towing: length exceeds 200m or breadth exceeds 25m',
       'Tanker, Hazardous category C',
       'Passenger, No additional information'], dtype=object)

In [64]:
min_duration=10
path_to_write='E:/Acer backup 3/internships and jobs/UW/Reader grader and TA/Shima Abadi/Data/Benchmark_data/'

In [65]:
reload(fn)

<module 'functions' from 'C:\\Users\\khirod\\Documents\\GitHub\\Ocean-data-analysis\\functions.py'>

In [66]:
def get_benchmark_data(df,hydrophone_idx,inner_rad,outer_rad,min_duration,fmin=10,fmax=90,break_dur=True,get_acoustic=True,get_spectrogram=True):
    
    if hydrophone_idx==4:
        hydro='Oregon_slope'
    elif hydrophone_idx==5:
        hydro='Southern_hydrate'
    suffix=str(inner_rad)+'_'+ str(outer_rad)+'.csv'
    ais=fn.choose_df_slope_hydrate(df,hydrophone_idx)
    isolated_ships=fn.get_isolated_ships(ais,inner_rad,outer_rad,min_duration)
    isolated_ships.to_csv(path_to_write+hydro+'/isolated_ships_' + suffix)
    isolated_ais=fn.isolated_ais(ais,isolated_ships,inner_rad)
    isolated_ais.to_csv(path_to_write+hydro+'/isolated_ais_'+ suffix)
    if break_dur:
        isolated_ais_10m=fn.break_duration(isolated_ships)
        isolated_ais_10m.to_csv(path_to_write+hydro+'/isolated_ais_10m_'+ suffix)
    
    if get_acoustic==True:
        print('Saving wav files')
        for i in tqdm(range(len(isolated_ships))):
            min_time=isolated_ships.start_time[i]
            max_time=isolated_ships.end_time[i]
            hydrophone_data=fn.get_acoustic(hydrophone_idx,min_time,max_time,fmin=fmin,fmax=fmax)
            if hydrophone_data==None:
                # print('data trace is none. Continuing to next')
                pass
            else:
                hydrophone_data.wav_write(filename=path_to_write+hydro+'/Audio'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.wav')
    if get_spectrogram==True:
        print('Saving spectrograms')
        for i in tqdm(range(len(isolated_ships))):
            min_time=isolated_ships.start_time[i]
            max_time=isolated_ships.end_time[i]
            spectrogram=fn.get_spectrogram_data(hydrophone_idx,min_time,max_time,fmin=fmin,fmax=fmax)
            if (spectrogram==None):
                # print('data trace is none. Continuing to next')
                pass
            else:
                if (spectrogram.values.shape[0]<2) | (spectrogram.values.shape[1]<2):
                    pass
                else:
                    #spectrogram.visualize(save_spec=True,plot_spec=False,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.png')
                    ooiplt.plot_spectrogram(spectrogram,plot=False,save=True,fmin=fmin,fmax=fmax,filename=path_to_write+hydro+'/Spectrogram'+'/isolated_ships_'+str(inner_rad)+'_'+ str(outer_rad)+'_'+str(i)+'.png')
    return isolated_ships,isolated_ais

## Creating benchmark data for Oregon slope

### Depth=2.9
### Oregon slope= 'HYSB1'

### inner rad=2.9, outer_rad=2.9, outer_rad=5.8

In [67]:
inner_rad= 6
outer_rad1=12
outer_rad2=24

In [68]:
d1,d2=get_benchmark_data(df,4,inner_rad,outer_rad1,min_duration,break_dur=False,get_acoustic=False,get_spectrogram=False)

 Max distance: 57.89 and Min distance: 0.0


100%|███████████████████████████████████████████████████████████████████████████████| 498/498 [00:04<00:00, 121.30it/s]


In [69]:
print(d1['VESSEL TYPE'].value_counts(dropna=False),'\n',d1['len_of_recording'].sum())

Cargo, all ships of this type         180
NaN                                   177
Other Type, all ships of this type     69
Towing                                 24
Tanker, all ships of this type         20
Fishing                                10
Pleasure Craft                          6
Sailing                                 5
Not available (default)                 2
Passenger, all ships of this type       1
Tanker, No additional information       1
Spare - Local Vessel                    1
Cargo, Hazardous category A             1
Cargo, No additional information        1
Name: VESSEL TYPE, dtype: int64 
 35560.433333333334


In [70]:
d1,d2=get_benchmark_data(df,4,inner_rad,outer_rad2,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 57.89 and Min distance: 0.0


100%|███████████████████████████████████████████████████████████████████████████████| 428/428 [00:03<00:00, 133.00it/s]


{'MMSI': 248264000, 'start_time': Timestamp('2017-11-19 05:17:12'), 'end_time': Timestamp('2017-11-19 05:27:12'), 'VESSEL TYPE': nan, 'len_of_recording': 10.0}
{'MMSI': 416498000, 'start_time': Timestamp('2019-07-23 11:55:50'), 'end_time': Timestamp('2019-07-23 12:05:50'), 'VESSEL TYPE': 'Cargo, all ships of this type', 'len_of_recording': 10.0}


In [71]:
print(d1['VESSEL TYPE'].value_counts(dropna=False),'\n',d1['len_of_recording'].sum())

NaN                                   155
Cargo, all ships of this type         152
Other Type, all ships of this type     60
Towing                                 19
Tanker, all ships of this type         17
Fishing                                 8
Pleasure Craft                          5
Sailing                                 4
Not available (default)                 3
Passenger, all ships of this type       1
Tanker, No additional information       1
Spare - Local Vessel                    1
Cargo, Hazardous category A             1
Cargo, No additional information        1
Name: VESSEL TYPE, dtype: int64 
 24582.9


In [72]:
d1.head()

,MMSI,start_time,end_time,count,VESSEL TYPE,len_of_recording
0,211331640,2020-01-24 11:10:41,2020-01-24 11:28:00,11,"Cargo, all ships of this type",17.316667
1,212715000,2021-04-15 21:02:14,2021-04-15 21:17:24,14,"Cargo, all ships of this type",15.166667
2,229711000,2018-02-06 08:42:59,2018-02-06 08:57:10,13,"Cargo, all ships of this type",14.183333
3,235009240,2020-06-24 00:44:23,2020-06-24 00:58:42,14,"Cargo, all ships of this type",14.316667
4,235009240,2021-09-05 22:36:16,2021-09-05 22:56:33,19,"Cargo, all ships of this type",20.283333


## Creating benchmark data for Southern hydrate

### Depth=772
### Southern hydrate= 'HYS14'

### inner rad=0.772, outer_rad=0.772, outer_rad=1.544

In [73]:
inner_rad= 2
outer_rad1=4
outer_rad2=8

In [74]:
d1,d2=get_benchmark_data(df,5,inner_rad,outer_rad1,min_duration,break_dur=False,get_acoustic=False,get_spectrogram=False)

 Max distance: 76.26 and Min distance: 0.0


100%|█████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 130.47it/s]


In [75]:
print(d1['VESSEL TYPE'].value_counts(dropna=False),'\n',d1['len_of_recording'].sum())

NaN                                   48
Other Type, all ships of this type    17
Fishing                                4
Towing                                 1
Tug                                    1
Pleasure Craft                         1
Name: VESSEL TYPE, dtype: int64 
 27208.816666666666


In [35]:
d1,d2=get_benchmark_data(df,2,inner_rad,outer_rad2,min_duration,get_acoustic=False,get_spectrogram=False)

 Max distance: 76.26 and Min distance: 0.0


100%|█████████████████████████████████████████████████████████████████████████████████| 93/93 [00:00<00:00, 121.69it/s]


{'MMSI': 367466760, 'start_time': Timestamp('2017-08-27 00:53:21'), 'end_time': Timestamp('2017-08-27 01:03:21'), 'VESSEL TYPE': 'Fishing', 'len_of_recording': 10.0}


In [76]:
print(d1['VESSEL TYPE'].value_counts(dropna=False),'\n',d1['len_of_recording'].sum())

NaN                                   48
Other Type, all ships of this type    17
Fishing                                4
Towing                                 1
Tug                                    1
Pleasure Craft                         1
Name: VESSEL TYPE, dtype: int64 
 27208.816666666666


In [77]:
reload(fn)

def compare_hydrophone_records_slope_hydrate(df,hydro_idx,inner_rad,outer_rad,min_duration,break_dur=True):
    print(colored('Comparative view with minimum duration: '+ str(min_duration),'green',attrs=['bold']))
    table=pd.DataFrame(columns=['hydrophone_idx','inner_rad','outer_rad','no_unique_ships','no_unique_vessels','no_instances'])
    for i_r in inner_rad:
        for o_r in outer_rad:
            print(o_r)
            if i_r<=o_r:
                temp_dict=dict()
                ais=fn.choose_df_slope_hydrate(df,hydro_idx,verbose=False)
                temp_df1=fn.get_isolated_ships(ais,i_r,o_r,min_duration)
                if break_dur:
                    temp_df2=fn.break_duration(temp_df1)
                    temp_dict['no_10m_durations']=len(temp_df2)
#                 print(len(temp_df1))
                temp_dict['hydrophone_idx']=hydro_idx
                temp_dict['inner_rad']=i_r
                temp_dict['outer_rad']=o_r
                temp_dict['no_unique_ships']=temp_df1['MMSI'].nunique()
                temp_dict['no_unique_vessels']=temp_df1['VESSEL TYPE'].nunique()
                temp_dict['no_instances']=len(temp_df1)
                temp_dict['len_of_recordings']=temp_df1.len_of_recording.sum()
                
                table=table.append(temp_dict,ignore_index=True)
    return table


In [78]:
df.head()
vessels=df[['MMSI','VESSEL TYPE']].drop_duplicates(subset=['MMSI'])
vessels

,MMSI,VESSEL TYPE
0,636092819,NaN
78,351819000,NaN
109,563052600,NaN
125,636092872,NaN
130,373054000,NaN
...,...,...
706707,372997000,"Cargo, all ships of this type"
706857,215134000,"Cargo, all ships of this type"
707518,308808000,"Cargo, all ships of this type"
707520,352948000,"Cargo, all ships of this type"


## Comparison of various options

In [80]:
inner_rad=[6]
outer_rad=[12,24]
min_duration=10
hydro_idx=4
table=compare_hydrophone_records_slope_hydrate(df,hydro_idx,inner_rad,outer_rad,min_duration,break_dur=True)
table

Comparative view with minimum duration: 10
12
24
{'MMSI': 248264000, 'start_time': Timestamp('2017-11-19 05:17:12'), 'end_time': Timestamp('2017-11-19 05:27:12'), 'VESSEL TYPE': nan, 'len_of_recording': 10.0}
{'MMSI': 416498000, 'start_time': Timestamp('2019-07-23 11:55:50'), 'end_time': Timestamp('2019-07-23 12:05:50'), 'VESSEL TYPE': 'Cargo, all ships of this type', 'len_of_recording': 10.0}


,hydrophone_idx,inner_rad,outer_rad,no_unique_ships,no_unique_vessels,no_instances,no_10m_durations,len_of_recordings
0,4.0,6.0,12.0,211.0,13.0,498.0,3836.0,35560.433333
1,4.0,6.0,24.0,182.0,13.0,428.0,2699.0,24582.900000


In [81]:
inner_rad=[2]
outer_rad=[4,8]
min_duration=10
hydro_idx=5
table=compare_hydrophone_records_slope_hydrate(df,hydro_idx,inner_rad,outer_rad,min_duration,break_dur=True)
table

Comparative view with minimum duration: 10
4
8
{'MMSI': 367466760, 'start_time': Timestamp('2017-08-27 00:53:21'), 'end_time': Timestamp('2017-08-27 01:03:21'), 'VESSEL TYPE': 'Fishing', 'len_of_recording': 10.0}


,hydrophone_idx,inner_rad,outer_rad,no_unique_ships,no_unique_vessels,no_instances,no_10m_durations,len_of_recordings
0,5.0,2.0,4.0,20.0,5.0,72.0,2757.0,27208.816667
1,5.0,2.0,8.0,20.0,5.0,93.0,2688.0,26396.233333


In [56]:
# reload(fn)

# print(colored('Overlapping of all hydrophones with minimum number of timestamps: '+ str(min_duration),'green',attrs=['bold']))
# table2=pd.DataFrame(columns=['inner_rad','outer_rad','no_unique_ships','no_unique_vessels','no_records'])

# for i_r in inner_rad:
#     for o_r in outer_rad:
#         combined_df=pd.DataFrame()
#         for i in range(1,4,1):
#             temp_dict=dict()
#             ais=fn.choose_df(df,i,verbose=False)
#             d1=fn.get_isolated_ships(ais,i_r,o_r,min_duration)
#             combined_df=pd.concat([combined_df,d1])

#         temp_dict['inner_rad']=i_r
#         temp_dict['outer_rad']=o_r
#         temp_dict['no_unique_ships']=combined_df['MMSI'].nunique()
#         temp_dict['no_unique_vessels']=combined_df['VESSEL TYPE'].nunique()
#         temp_dict['no_records']=len(combined_df)
#         table2=table2.append(temp_dict,ignore_index=True)
# table2
        